# The traveltime lookup table

This tutorial will cover the basic ideas and definitions behind the traveltime lookup table, as well as showing the user how to create their own.

***

## Contents
* [Defining the underlying 3-D grid](#Defining-the-underlying-3-D-grid)
* [Creating a LUT](#Creating-an-instance-of-the-LUT-class)
* [Computing traveltimes](#Computing-traveltimes)

***

In order to reduce computational costs during runtime, we pre-compute traveltime
lookup tables (LUTs). These LUTs contain P- and S-phase traveltimes for each station in the
network to every point in a 3-D grid. This grid spans the volume of interest, herein termed
the coalescence volume, within which QuakeMigrate will search for events.

## Defining the underlying 3-D grid

Before we can create our traveltime lookup table, we have to define the underlying 3-D grid which spans the volume of interest.

### Coordinate projections

First, we choose a pair of projections to represent the input coordinate space and the Cartesian grid space. We do this using the Python interface with the PROJ library, pyproj. It is important to think about which projection is best suited to your particular study region. 

We use here the WGS84 reference ellipsoid (used as standard by the Global Positioning System) as our input space and the Lambert Conformal Conic projection to form our Cartesian space. The units of the Cartesian space are specified as metres. The values used in the LCC projection are for a study region in northern Borneo.

In [ ]:
from pyproj import Proj

cproj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gproj = Proj("+proj=lcc +lon_0=116.75 +lat_0=6.25 +lat_1=4.0 +lat_2=7.5 +datum=WGS84 +units=m +no_defs") 

### Geographical location and spatial extent

In order to geographically situate our lookup table, we define two points, herein called the lower-left and upper-right corners. By default, we work in a depth-positive frame (i.e. positive down or left-handed coordinate system) and use metres. It is in theory possible to run QuakeMigrate with distances measured in kilometres, as long as the user specifies this requirement when defining the grid projection.

This schematic shows the relative positioning of the two corners:

![title](img/LUT_definition.png)

The final piece of information required to fully define the grid on which we will calculate traveltimes is the size (in each dimension, x, y, z) of a cell. The LUT class will automatically find the number of cells required in each dimension to span the specified geographical region. If a cell dimension doesn't fit into the corresponding grid dimension an integer number of times, the location of the upper-right corner is shifted to accommodate an additional cell.

In [ ]:
ll_corner = [116.075, 5.573, -1750]
ur_corner = [117.426, 6.925, 27750]
cell_size = [500., 500., 500.]

## Creating an instance of the LUT class

We are now ready to create an instance of the LUT class, which we can then populate with traveltimes. We import the LUT module, which contains two submodules: lut.py, which contains the LUT class definition; and create_lut.py, which contains a suite of utility functions to compute traveltimes and populate a LUT object.

In [ ]:
import QMigrate.lut as qlut

# --- Create a new LUT ---
lut = qlut.LUT(ll_corner=ll_corner,
               ur_corner=ur_corner,
               cell_size=cell_size,
               grid_proj=gproj,
               coord_proj=cproj)

## Computing traveltimes

We have bundled a few methods of computing traveltimes into QuakeMigrate.

In all cases we will make use of the I/O module, so let's import that first:

In [ ]:
import QMigrate.io as qio

#stations = qio.stations("/path/to/station_file")

### Homogeneous velocity model
Simply calculates the straight line traveltimes between stations and points in the grid.

In [ ]:
# --- Homogeneous LUT generation ---
#qlut.compute(lut, stations, method="homogeneous", vp=5000., vs=3000.)

### Fast-marching method
The fast-marching method implicitly tracks the evolution of the wavefront. See Rawlinson & Sambridge (2005) for more details.

In [ ]:
# --- skfmm LUT generation ---
#vmod = qio.read_vmodel("/path/to/vmodel_file")
#qlut.compute(lut, stations, method="1dfmm", vmod=vmod)

### NonLinLoc style 2-D sweep
Uses the Eikonal solver from NonLinLoc under the hood to generate a traveltime grid for the 2-D slice that passes through the station and the point in the grid furthest away from that station. This slice is then "swept" using a bilinear interpolation scheme to produce a 3-D traveltime grid. This has the benefit of being able to include stations outside of the volume of interest, without having to increase the size of the grid.

In [ ]:
# --- NLLoc sweep LUT generation ---
#vmod = qio.read_vmodel("/path/to/vmodel_file")
#qlut.compute(lut, stations, method="1dsweep", vmod=vmod, block_model=True)

### Other formats
It is also easy to import traveltime lookup tables generated by other means. We have provided a parser for lookup tables in the NonLinLoc format (`qlut.read_nlloc()`). It is straightforward to adapt this code to read any other traveltime lookup table, so long as it is stored as an array. Create an instance of the LUT class with the correct grid dimensions, then add the traveltime arrays (in C-order) to the _maps_ dictionary.

Finally, save the lookup table to a file. We pickle the object by default.

In [ ]:
# --- Save LUT ---
#lut.save("/path/to/output/lut")